Setup MODIN

In [1]:
import os

os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

import modin.pandas as pd

Import Libraries

In [2]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 80
pd.options.display.max_rows = 80
import seaborn as sns
%matplotlib inline
sns.set()

In [3]:
data = pd.read_pickle('cooked.pkl')

distributed.diskutils - INFO - Found stale lock file and directory '/home/mlop3n/PycharmProjects/tps_june/notebooks/dask-worker-space/worker-gayn3hdp', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mlop3n/PycharmProjects/tps_june/notebooks/dask-worker-space/worker-gn90cbwb', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mlop3n/PycharmProjects/tps_june/notebooks/dask-worker-space/worker-fpx9e_k1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mlop3n/PycharmProjects/tps_june/notebooks/dask-worker-space/worker-qye92c2v', purging


In [4]:
data.info()

<class 'modin.pandas.dataframe.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 81 columns):
 #   Column        Non-Null Count    Dtype  
---  ------------  ----------------  -----  
 0   F_1_0         981603 non-null   float32
 1   F_1_1         981784 non-null   float32
 2   F_1_2         981992 non-null   float32
 3   F_1_3         981750 non-null   float32
 4   F_1_4         981678 non-null   float32
 5   F_1_5         981911 non-null   float32
 6   F_1_6         981867 non-null   float32
 7   F_1_7         981872 non-null   float32
 8   F_1_8         981838 non-null   float32
 9   F_1_9         981751 non-null   float32
 10  F_1_10        982039 non-null   float32
 11  F_1_11        981830 non-null   float32
 12  F_1_12        981797 non-null   float32
 13  F_1_13        981602 non-null   float32
 14  F_1_14        981961 non-null   float32
 15  F_2_0         1000000 non-null  uint8
 16  F_2_1         1000000 non-null  uint8
 17  F_2_2         1000000 non-n

In [5]:
dm_1 = data[data.missing_cols==1]
dc = data[data.missing_cols==0]

Make datasets for single column learning<br>
 - Methods possible other than imputation:
 - PCA(dimensionality reduction)
 - feature elimination

Initial Steps<br>
 - [F]_clean stores indexes where each column is not missing
 - Separate them based on number of missing columns in each row

In [6]:
missing_ = data.isna().sum(axis=0)
missing_features = np.array(missing_[missing_ > 0].index)

    missing_features = array(['F_1_0', 'F_1_1', 'F_1_2', 'F_1_3', 'F_1_4', 'F_1_5', 'F_1_6',
       'F_1_7', 'F_1_8', 'F_1_9', 'F_1_10', 'F_1_11', 'F_1_12', 'F_1_13',
       'F_1_14', 'F_3_0', 'F_3_1', 'F_3_2', 'F_3_3', 'F_3_4', 'F_3_5',
       'F_3_6', 'F_3_7', 'F_3_8', 'F_3_9', 'F_3_10', 'F_3_11', 'F_3_12',
       'F_3_13', 'F_3_14', 'F_3_15', 'F_3_16', 'F_3_17', 'F_3_18',
       'F_3_19', 'F_3_20', 'F_3_21', 'F_3_22', 'F_3_23', 'F_3_24',
       'F_4_0', 'F_4_1', 'F_4_2', 'F_4_3', 'F_4_4', 'F_4_5', 'F_4_6',
       'F_4_7', 'F_4_8', 'F_4_9', 'F_4_10', 'F_4_11', 'F_4_12', 'F_4_13',
       'F_4_14'], dtype=object)

In [21]:
F_clean = {f:data[data[f].isna()==False].index for f in missing_features}
F_dirty = {f:data[data[f].isna()].index for f in missing_features}

In [10]:
missing_.sort_values(inplace=True)

In [ ]:
missing_[missing_>0].plot(kind='bar', figsize=(20,20),logy=True, fontsize=14)

#### Missing Data Distribution
![Image](missing_data_distribution.png)

#### Densities of the least missing features
##### F_1_10 and F_4_4
![Image](F_1_0density.png)![Image](F_4_4.png)

In [13]:
training_order = missing_.index

 - Developed Training Order
 